<a href="https://colab.research.google.com/github/illhyhl1111/SNU_ML2019/blob/master/LogicGate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Logic Gate Example

In [0]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np

#### class Separator
- variables : a,b,c (initialize with Norm(0,1))
- func : activation functions (such as sigmoid, relu, except softmax)
- inputs : x,y point
- output : z_

In [0]:
class Separator(nn.Module):
  def __init__(self, func = None):
    super(Separator, self).__init__()
    self.a = torch.nn.Parameter(torch.Tensor([np.random.normal()]))
    self.b = torch.nn.Parameter(torch.Tensor([np.random.normal()]))
    self.c = torch.nn.Parameter(torch.Tensor([np.random.normal()]))
    if func is not None:
      self.func = func
    else:
      self.func = torch.sigmoid
    
  def forward(self,x,y):
    val = self.func(self.a * x + self.b * y - self.c)
    return val

#### dataset DataGenerator
- dataset : (x,y), validate
- x,y are picked from {0,1} with gaussian error N(0,0.5^2)

In [0]:
class DataGenerator(Dataset):
  
  def __init__(self,type_,length,custom=None):
    self.length=length
    if type_ == 'and':
      self.val_l = [0,0,0,1]
    elif type_ == 'or':
      self.val_l = [0,1,1,1]
    elif type_ == 'xor':
      self.val_l = [0,1,1,0]
    elif type_ == 'custom' and custom is not None:
      self.val_l = custom
    else:
      self.val_l = [0,0,0,0]
      
    self.dataset = []
    for i in range(length):
      x = np.random.normal(i%2,0.25)
      y = np.random.normal((i//2)%2,0.25)
      val = self.val_l[i%4]
      self.dataset.append((x,y,val))
      
  def __len__(self):
    return self.length
            
  def __getitem__(self,idx):
    x,y,val = self.dataset[idx]
    return (torch.Tensor([x]),torch.Tensor([y]), torch.Tensor([val]))

#### Hyperparameters & Dataset


In [0]:
DATASET = DataGenerator('and',1000)
LEARNING_RATE = 0.01
BATCH_SIZE = 20
NUM_EPOCHES = 20
NUM_WORKERS = 4

params = {
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'num_workers': 4,
}

dataloader = DataLoader(DATASET,**params)
model = Separator().cuda()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

#### Training part

In [0]:
for epoch in range(NUM_EPOCHES):
  tot_loss = 0
  for x, y, val in dataloader:
    x = x.cuda()
    y = y.cuda()
    val = val.cuda()
    optimizer.zero_grad()
    val_ = model(x,y)
    loss = torch.sum(torch.pow(val-val_,2))
    
    loss.backward()
    optimizer.step()
    tot_loss+=loss.item()
  print("{:.5f}".format(tot_loss/len(DATASET)))
  print(model(0,0).item()>0.5,model(1,0).item()>0.5,model(0,1).item()>0.5,model(1,1).item()>0.5)

0.39429
True True True True
0.30597
True True False False
0.25880
False False False False
0.22200
False False False False
0.19204
False False False False
0.16892
False False False False
0.15135
False False False True
0.13783
False False False True
0.12728
False False False True
0.11857
False False False True
0.11144
False False False True
0.10559
False False False True
0.10060
False False False True
0.09629
False False False True
0.09234
False False False True
0.08892
False False False True
0.08585
False False False True
0.08318
False False False True
0.08073
False False False True
0.07850
False False False True
